In [ ]:
from pyspark.sql import SparkSession,functions as F
from pyspark import StorageLevel
from pyspark import SparkConf
from pyspark import SparkConf
import sys 
sys.path.append("/home/john_user/pyspark_learning/code/")

conf = SparkConf()

conf.set("spark.app.name", "ComprehensiveSparkJob") 
conf.set("spark.master", "local[*]")                
conf.set("spark.driver.memory", "4g")      
conf.set("spark.driver.cores", "1")                 
conf.set("spark.ui.port", "4040")                

# Executor Settings
conf.set("spark.executor.memory", "2g")      
conf.set("spark.executor.cores", "2")               
conf.set("spark.executor.instances", "3")         


conf.set("spark.default.parallelism", "6")         
conf.set("spark.sql.shuffle.partitions", "6")       
conf.set("spark.task.cpus", "1")  

# Data Handling Settings
conf.set("spark.memory.fraction", "0.8")            
conf.set("spark.memory.storageFraction", "0.5")     

# Serialization Settings
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")  
conf.set("spark.kryo.registrationRequired", "true")  
conf.set("spark.kryo.classesToRegister", "org.apache.spark.sql.Row")  

# Debugging and Logging
conf.set("spark.eventLog.enabled", "true")           
conf.set("spark.eventLog.dir", "/root/spark_log/spark-events/")  
conf.set("spark.history.fs.logDirectory", "/root/spark_log/spark-history/")  


In [ ]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [ ]:
input_json_file_path = "/root/docker_dataset/h1b_data/"  
file_pattern = r'.*\.(csv|xlsx)$'
input_file_list  = rfl(input_json_file_path,file_pattern)
# parquet_file_path = "/root/docker_dataset/json_splits/table_example_1.parquet"



In [ ]:

def func_cache_dft_data(dataframe) :
    dataframe.persist(StorageLevel.MEMORY_AND_DISK)

def func_json_dataframe (input_file:str) :
    return  spark.read.option("multiline", "true").json(input_file)

def func_repartion_dataframe(dataframe, repartion_count) :
    return dataframe.repartition(repartion_count)
 

def func_explode_based_on_one_key (input_dataframe,explode_key:str , alias_of_column :str)  :
    return  input_dataframe.select(F.explode(F.col(explode_key)).alias(alias_of_column))

def func_explode_based_on_depper_root_key (input_dataframe,explode_key:list  , drop_column :str)  :
    """ Always try to assemeble  he json key  based on the root level  in heirarchial order like root -->values-->payload-->commits then the explode_key   value should be [root,values,payload,commits]"""
    column_level_analysis = (".").join(explode_key)
    return  input_dataframe.select(f"{explode_key[0]}.*",F.explode_outer(F.col(column_level_analysis)).alias(column_level_analysis.replace(".","_"))).drop(drop_column)


def func_get_rdd_num_partitions( dataframe) :
    return dataframe.rdd.getNumPartitions()

def func_write_data_to_parquet ( file_path, dataframe,write_mode) :
    dataframe.write.parquet(file_path,mode=write_mode)


def func_read_parquet_file(file_path) :
    return spark.read.parquet(file_path)

def func_stop_spark () :
      spark.stop()


def func_remane_column_names (dataframe) :
    column_list = [F.col(c).alias(c.replace('.', '_')) for c in dataframe.columns]
    for   values in column_list:
        print(values)
    return  dataframe.select([F.col(c).alias(c.replace('.', '_')) for c in dataframe.columns])


def  func_limit_rows_dataframe (dataframe, limit_row_count) :
    return dataframe. limit(limit_row_count)

def  finc_count_dft_rows(dataframe):
     return dataframe.count()



In [ ]:

def func_main () :
     processed_dft_row_count= 0 
     for file_list in input_file_list :
         big_dpart_dft = func_json_dataframe (input_json_file_path+file_list)
         big_dpart_dft = func_repartion_dataframe(big_dpart_dft,100)
         big_dpart_dft_json_normalised = func_explode_based_on_one_key (big_dpart_dft,"values","table_example")
         big_dpart_dft_json_normalised = func_explode_based_on_depper_root_key(big_dpart_dft_json_normalised,["table_example","entities"],"entities")
         func_cache_dft_data(big_dpart_dft_json_normalised)
         print(func_get_rdd_num_partitions(big_dpart_dft_json_normalised))
         big_dpart_dft_json_norm_renamed  = func_remane_column_names(big_dpart_dft_json_normalised)
         processed_dft_row_count += finc_count_dft_rows(big_dpart_dft_json_norm_renamed)
         func_write_data_to_parquet(parquet_file_path,big_dpart_dft_json_norm_renamed,"append")
     return processed_dft_row_count

In [ ]:
try :
    # json_input_row_count  = func_main () 
    big_dpart_parquet = func_read_parquet_file(parquet_file_path)
    # parquet_row_count  = finc_count_dft_rows(big_dpart_parquet)
    # big_dpart_parquet = func_limit_rows_dataframe(big_dpart_parquet,100)
    big_dpart_parquet.show()
except  Exception as error :
    spark.stop()
    raise error 

In [ ]:
db_url = "jdbc:postgresql://host.docker.internal:5432/crime_data_la"
db_properties = {
    "user": "john_user",
    "password": "abc@12345"
}

table_name = "crime_data.transactions_data"
df_table = spark.read.jdbc(url=db_url, table=table_name, properties=db_properties)
df_table.show()


In [ ]:
data_type  = "cast ( employer_poc_phone as string ) as employer_poc_phone,cast ( new_employment as Long ) as new_employment,cast ( continued_employment as Long ) as continued_employment,cast ( agent_attorney_phone as decimal ) as agent_attorney_phone,cast ( agent_attorney_phone_ext as decimal ) as agent_attorney_phone_ext,cast ( change_previous_employment as Long ) as change_previous_employment,cast ( pw_other_year as decimal ) as pw_other_year,cast ( new_concurrent_employment as Long ) as new_concurrent_employment,cast ( employer_poc_phone_ext as decimal ) as employer_poc_phone_ext,cast ( total_worksite_locations as decimal ) as total_worksite_locations,cast ( employer_phone_ext as decimal ) as employer_phone_ext,cast ( worksite_workers as decimal ) as worksite_workers,cast ( naics_code as Long ) as naics_code,cast ( change_employer as Long ) as change_employer,cast ( amended_petition as Long ) as amended_petition,cast ( original_cert_date as date ) as original_cert_date,cast ( received_date as date ) as received_date,cast ( decision_date as date ) as decision_date,cast ( begin_date as date ) as begin_date,cast ( end_date as date ) as end_date,cast ( wage_rate_of_pay_from as decimal ) as wage_rate_of_pay_from,cast ( wage_rate_of_pay_to as decimal ) as wage_rate_of_pay_to,cast ( total_worker_positions as Long ) as total_worker_positions,cast ( employer_poc_email as string ) as employer_poc_email,cast ( agent_representing_employer as string ) as agent_representing_employer,cast ( agent_attorney_last_name as string ) as agent_attorney_last_name,cast ( agent_attorney_first_name as string ) as agent_attorney_first_name,cast ( agent_attorney_middle_name as string ) as agent_attorney_middle_name,cast ( agent_attorney_address1 as string ) as agent_attorney_address1,cast ( agent_attorney_address2 as string ) as agent_attorney_address2,cast ( agent_attorney_city as string ) as agent_attorney_city,cast ( agent_attorney_state as string ) as agent_attorney_state,cast ( agent_attorney_postal_code as string ) as agent_attorney_postal_code,cast ( agent_attorney_country as string ) as agent_attorney_country,cast ( agent_attorney_province as string ) as agent_attorney_province,cast ( agent_attorney_email_address as string ) as agent_attorney_email_address,cast ( lawfirm_name_business_name as string ) as lawfirm_name_business_name,cast ( state_of_highest_court as string ) as state_of_highest_court,cast ( name_of_highest_state_court as string ) as name_of_highest_state_court,cast ( secondary_entity as string ) as secondary_entity,cast ( secondary_entity_business_name as string ) as secondary_entity_business_name,cast ( worksite_address1 as string ) as worksite_address1,cast ( worksite_address2 as string ) as worksite_address2,cast ( worksite_city as string ) as worksite_city,cast ( worksite_county as string ) as worksite_county,cast ( worksite_state as string ) as worksite_state,cast ( worksite_postal_code as string ) as worksite_postal_code,cast ( wage_unit_of_pay as string ) as wage_unit_of_pay,cast ( pw_unit_of_pay as string ) as pw_unit_of_pay,cast ( pw_tracking_number as string ) as pw_tracking_number,cast ( pw_wage_level as string ) as pw_wage_level,cast ( pw_oes_year as string ) as pw_oes_year,cast ( pw_other_source as string ) as pw_other_source,cast ( pw_survey_publisher as string ) as pw_survey_publisher,cast ( pw_survey_name as string ) as pw_survey_name,cast ( agree_to_lc_statement as string ) as agree_to_lc_statement,cast ( h_1b_dependent as string ) as h_1b_dependent,cast ( willful_violator as string ) as willful_violator,cast ( support_h1b as string ) as support_h1b,cast ( statutory_basis as string ) as statutory_basis,cast ( appendix_a_attached as string ) as appendix_a_attached,cast ( public_disclosure as string ) as public_disclosure,cast ( preparer_last_name as string ) as preparer_last_name,cast ( preparer_first_name as string ) as preparer_first_name,cast ( preparer_middle_initial as string ) as preparer_middle_initial,cast ( preparer_business_name as string ) as preparer_business_name,cast ( preparer_email as string ) as preparer_email,cast ( case_number as string ) as case_number,cast ( case_status as string ) as case_status,cast ( visa_class as string ) as visa_class,cast ( job_title as string ) as job_title,cast ( soc_code as string ) as soc_code,cast ( soc_title as string ) as soc_title,cast ( full_time_position as string ) as full_time_position,cast ( employer_name as string ) as employer_name,cast ( trade_name_dba as string ) as trade_name_dba,cast ( employer_address1 as string ) as employer_address1,cast ( employer_address2 as string ) as employer_address2,cast ( employer_city as string ) as employer_city,cast ( employer_state as string ) as employer_state,cast ( employer_postal_code as string ) as employer_postal_code,cast ( employer_country as string ) as employer_country,cast ( employer_province as string ) as employer_province,cast ( employer_phone as string ) as employer_phone,cast ( employer_poc_last_name as string ) as employer_poc_last_name,cast ( employer_poc_first_name as string ) as employer_poc_first_name,cast ( employer_poc_middle_name as string ) as employer_poc_middle_name,cast ( employer_poc_job_title as string ) as employer_poc_job_title,cast ( employer_poc_address1 as string ) as employer_poc_address1,cast ( employer_poc_address2 as string ) as employer_poc_address2,cast ( employer_poc_city as string ) as employer_poc_city,cast ( employer_poc_state as string ) as employer_poc_state,cast ( employer_poc_postal_code as string ) as employer_poc_postal_code,cast ( employer_poc_country as string ) as employer_poc_country,cast ( employer_poc_province as string ) as employer_poc_province,cast ( prevailing_wage as decimal ) as prevailing_wage"

In [ ]:
csv_path_root  = '/root/docker_dataset/h1b_data/'
input = [
                "Combined_LCA_Disclosure_Data_FY2020.csv",
                "Combined_LCA_Disclosure_Data_FY2021.csv",
                "Combined_LCA_Disclosure_Data_FY2022.csv",
                "Combined_LCA_Disclosure_Data_FY2023.csv",
                "Combined_LCA_Disclosure_Data_FY2024.csv"
            ]
formatted_data_type = [f'{item.strip()}' for item in data_type.split(',')]
print(formatted_data_type)
for  value in input  :
        csv_path  = csv_path_root +value
        df_csv = spark.read.option("header", "true").option("inferSchema", "false").csv(csv_path)
        df_csv_updated = df_csv.selectExpr(
            formatted_data_type
        )
        df_csv.show()

In [ ]:
df_csv_updated.select(df_csv_updated["CASE_STATUS"]).show()
df_csv_updated.count()

In [ ]:
db_url = "jdbc:postgresql://host.docker.internal:5432/crime_data_la"
db_properties = {
    "user": "john_user",
    "password": "abc@12345"
}
mode = "overwrite"
table_name = "spark_project.h1b_data"
df_csv_updated.write.option('driver', 'org.postgresql.Driver').jdbc(url=db_url, table=table_name, properties=db_properties)
